In [1]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from keras import backend as K 
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

Using TensorFlow backend.
/Users/ewallner/.brew/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
max_caption_len = 3
vocab_size = 3

In [4]:
# Load one screenshot for each word, and turn them into digits 
images = []
for i in range(2):
    images.append(img_to_array(load_img('screenshot.jpg', target_size=(224, 224))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

In [5]:
html_input = np.array(
            [[[0., 0., 0.], #start
             [0., 0., 0.],
             [1., 0., 0.]],
             [[0., 0., 0.], #start <HTML>Hello World!</HTML>
             [1., 0., 0.],
             [0., 1., 0.]]])

next_words = np.array(
            [[0., 1., 0.], # <HTML>Hello World!</HTML>
             [0., 0., 1.]]) # end

In [6]:
# We extract information from the images
VGG = VGG16(weights='imagenet', include_top=True)
VGG.trainable = False
vgg_out = RepeatVector(max_caption_len)(VGG.output)

553467904/553467096 [==============================] - 88s 0us/step


In [7]:
# Extract information from the input seqence 
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(5, return_sequences=True)(language_input)

In [8]:
# Concatenate the information from the image and the input
decoder = concatenate([vgg_out, language_model])
# Extract information from the concatenated output
decoder = LSTM(5, return_sequences=False)(decoder)
# Predict which word comes next
decoder_output = Dense(vocab_size, activation='softmax')(decoder)
# Compile and run the neural network
model = Model(inputs=[VGG.input, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [9]:
model.fit([images, html_input], next_words, batch_size=2, shuffle=False, epochs=500)

Epoch 1/500
2/2 [==============================] - 9s 4s/step - loss: 1.0974
Epoch 2/500
2/2 [==============================] - 4s 2s/step - loss: 1.0809
Epoch 3/500
2/2 [==============================] - 4s 2s/step - loss: 1.0771
Epoch 4/500
2/2 [==============================] - 4s 2s/step - loss: 1.0739
Epoch 5/500
2/2 [==============================] - 4s 2s/step - loss: 1.0711
Epoch 6/500
2/2 [==============================] - 4s 2s/step - loss: 1.0685
Epoch 7/500
2/2 [==============================] - 4s 2s/step - loss: 1.0661
Epoch 8/500
2/2 [==============================] - 4s 2s/step - loss: 1.0637
Epoch 9/500
2/2 [==============================] - 4s 2s/step - loss: 1.0615
Epoch 10/500
2/2 [==============================] - 4s 2s/step - loss: 1.0593
Epoch 11/500
2/2 [==============================] - 4s 2s/step - loss: 1.0571
Epoch 12/500
2/2 [==============================] - 4s 2s/step - loss: 1.0550
Epoch 13/500
2/2 [==============================] - 4s 2s/step - loss: 1.

2/2 [==============================] - 5s 2s/step - loss: 0.8441
Epoch 107/500
2/2 [==============================] - 4s 2s/step - loss: 0.8420
Epoch 108/500
2/2 [==============================] - 4s 2s/step - loss: 0.8399
Epoch 109/500
2/2 [==============================] - 4s 2s/step - loss: 0.8378
Epoch 110/500
2/2 [==============================] - 4s 2s/step - loss: 0.8357
Epoch 111/500
2/2 [==============================] - 4s 2s/step - loss: 0.8336
Epoch 112/500
2/2 [==============================] - 4s 2s/step - loss: 0.8315
Epoch 113/500
2/2 [==============================] - 4s 2s/step - loss: 0.8295
Epoch 114/500
2/2 [==============================] - 4s 2s/step - loss: 0.8274
Epoch 115/500
2/2 [==============================] - 5s 2s/step - loss: 0.8254
Epoch 116/500
2/2 [==============================] - 4s 2s/step - loss: 0.8234
Epoch 117/500
2/2 [==============================] - 5s 2s/step - loss: 0.8214
Epoch 118/500
2/2 [==============================] - 4s 2s/step - 

Epoch 210/500
2/2 [==============================] - 6s 3s/step - loss: 0.7052
Epoch 211/500
2/2 [==============================] - 6s 3s/step - loss: 0.7045
Epoch 212/500
2/2 [==============================] - 5s 2s/step - loss: 0.7037
Epoch 213/500
2/2 [==============================] - 5s 2s/step - loss: 0.7030
Epoch 214/500
2/2 [==============================] - 4s 2s/step - loss: 0.7022
Epoch 215/500
2/2 [==============================] - 4s 2s/step - loss: 0.7015
Epoch 216/500
2/2 [==============================] - 4s 2s/step - loss: 0.7007
Epoch 217/500
2/2 [==============================] - 4s 2s/step - loss: 0.7000
Epoch 218/500
2/2 [==============================] - 4s 2s/step - loss: 0.6993
Epoch 219/500
2/2 [==============================] - 4s 2s/step - loss: 0.6986
Epoch 220/500
2/2 [==============================] - 4s 2s/step - loss: 0.6978
Epoch 221/500
2/2 [==============================] - 5s 2s/step - loss: 0.6971
Epoch 222/500
2/2 [==============================] -

Epoch 314/500
2/2 [==============================] - 5s 2s/step - loss: 0.5755
Epoch 315/500
2/2 [==============================] - 4s 2s/step - loss: 0.5733
Epoch 316/500
2/2 [==============================] - 4s 2s/step - loss: 0.5710
Epoch 317/500
2/2 [==============================] - 4s 2s/step - loss: 0.5688
Epoch 318/500
2/2 [==============================] - 4s 2s/step - loss: 0.5665
Epoch 319/500
2/2 [==============================] - 4s 2s/step - loss: 0.5642
Epoch 320/500
2/2 [==============================] - 4s 2s/step - loss: 0.5619
Epoch 321/500
2/2 [==============================] - 4s 2s/step - loss: 0.5595
Epoch 322/500
2/2 [==============================] - 4s 2s/step - loss: 0.5572
Epoch 323/500
2/2 [==============================] - 4s 2s/step - loss: 0.5548
Epoch 324/500
2/2 [==============================] - 4s 2s/step - loss: 0.5525
Epoch 325/500
2/2 [==============================] - 4s 2s/step - loss: 0.5501
Epoch 326/500
2/2 [==============================] -

Epoch 418/500
2/2 [==============================] - 4s 2s/step - loss: 0.3060
Epoch 419/500
2/2 [==============================] - 4s 2s/step - loss: 0.3033
Epoch 420/500
2/2 [==============================] - 4s 2s/step - loss: 0.3007
Epoch 421/500
2/2 [==============================] - 5s 2s/step - loss: 0.2982
Epoch 422/500
2/2 [==============================] - 5s 2s/step - loss: 0.2957
Epoch 423/500
2/2 [==============================] - 4s 2s/step - loss: 0.2930
Epoch 424/500
2/2 [==============================] - 4s 2s/step - loss: 0.2905
Epoch 425/500
2/2 [==============================] - 4s 2s/step - loss: 0.2880
Epoch 426/500
2/2 [==============================] - 5s 2s/step - loss: 0.2854
Epoch 427/500
2/2 [==============================] - 5s 2s/step - loss: 0.2828
Epoch 428/500
2/2 [==============================] - 5s 2s/step - loss: 0.2803
Epoch 429/500
2/2 [==============================] - 4s 2s/step - loss: 0.2777
Epoch 430/500
2/2 [==============================] -

In [10]:
start_token = [1., 0., 0.] # start
sentence = np.zeros((1, 3, 3)) # [[0,0,0], [0,0,0], [0,0,0]]
sentence[0][2] = start_token # place start in empty sentence

In [11]:
# Making the first prediction with the start token
first_word = model.predict([np.array([images[1]]), sentence])

In [12]:
# Put the second word in the sentence and make the final prediction
sentence[0][1] = start_token
sentence[0][2] = np.round(second_word)
third_word = model.predict([np.array([images[1]]), sentence])

NameError: name 'second_word' is not defined

In [ ]:
# Place the start token and two predictions in the sentence 

In [ ]:
sentence[0][0] = start_token
sentence[0][1] = np.round(second_word)
sentence[0][2] = np.round(third_word)

In [ ]:
vocabulary = ["start", "<HTML><center><H1>Hello World!</H1><center></HTML>", "end"]
for i in sentence:
    print(vocabulary[np.argmax[i]])